# CS5346 Mini-Project 2 - Data Processing
---

In [8]:
import numpy as np
import datetime as dt
import pandas as pd
import json

In [5]:
with open('Mini-project-2-data.json', "r", encoding='utf-8') as json_data:
    raw_data = json.load(json_data)

In [6]:
len(raw_data)

200000

## Dataset 1
__arXiv__
- Author ID
- Autho Name
- Year
- Venue
- Journal Name
- In Citations

In [78]:
temp = pd.DataFrame(raw_data)
arXiv = temp.loc[temp['venue'] == "ArXiv"]
arXiv = arXiv[["authors", "id", "year", "inCitations"]]
arXiv["inCitations"] = arXiv["inCitations"].apply(lambda x: len(x))
arXiv["year"] = arXiv["year"].apply(lambda x: int(x))

In [79]:
temp = arXiv.set_index(['id', 'year', 'inCitations'])
unnested_lst = []
for col in temp.columns:
    unnested_lst.append(temp[col].apply(pd.Series).stack())
result = pd.concat(unnested_lst, axis=1, keys=temp.columns)
result = result.reset_index()
result["author_id"] = result["authors"].apply(lambda x: x["ids"][0] if len(x["ids"]) > 0 else "")
result["author_name"] = result["authors"].apply(lambda x: x["name"])
result = result.drop(['level_3', 'authors'], axis = 1)

In [142]:
counting = result.groupby(['author_id']).size().reset_index(name='counts')
counting = counting.sort_values("counts", ascending=False)
counting = counting.loc[counting['counts'] >= 3]
counting = counting.author_id.as_matrix()

In [161]:
top_ten = result.loc[result['author_id'].isin(counting)]
top_ten = top_ten.groupby(['author_name', 'year']).size().reset_index(name='counts')
top_ten = top_ten.pivot_table(index='author_name', columns=['year'], values='counts', fill_value=0).reset_index()

In [162]:
top_ten.to_csv('obj1.csv', sep=',', encoding='utf-8', index=False)

## Dataset 2
__arXiv__
- Year
- Venue
- In Citations

In [120]:
temp = pd.DataFrame(raw_data)
arXiv = temp.loc[temp['venue'] == "ArXiv"]
arXiv = arXiv[["id", "year", "inCitations"]]
arXiv["year"] = arXiv["year"].apply(lambda x: int(x))
arXiv["inCitations"] = arXiv["inCitations"].apply(lambda x: len(x))
arXiv = arXiv.sort_values("inCitations", ascending=False).groupby("year").head(5).sort_values(["year", "inCitations"], ascending=[True, False])

In [121]:
arXiv.to_csv('obj2.csv', sep=',', encoding='utf-8', index=False)

## Dataset 3
__ICSE__
- Authors
- Venue
- In Citations
- Out Citations

In [193]:
temp = pd.DataFrame(raw_data)

def getNames(authors):
    names = ""
    for a in authors: 
        names = names + a["name"] + ", "
    return names[:-2]

ICSE = temp.loc[temp['venue'] == "ICSE"]
ICSE = ICSE[["authors", "year", "inCitations", "outCitations"]]
ICSE["year"] = ICSE["year"].apply(lambda x: int(x))
ICSE["inCitations"] = ICSE["inCitations"].apply(lambda x: len(x))
ICSE["outCitations"] = ICSE["outCitations"].apply(lambda x: len(x))
ICSE["authors"] = ICSE["authors"].apply(lambda x: len(x))
ICSE_year = ICSE.groupby(['year']).size().reset_index(name='counts').set_index('year')
ICSE_others = ICSE.groupby(['year']).sum().reset_index()
ICSE = ICSE_year.join(ICSE_others.set_index('year'))

In [194]:
ICSE.to_csv('obj3.csv', sep=',', encoding='utf-8')